In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.optimizers import Adam
from keras.regularizers import l2
from kerastuner.tuners import RandomSearch

In [3]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
column_names = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
dataset = pd.read_csv(url, header=None, names=column_names)

# Separate features and target
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [4]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units1', min_value=64, max_value=512, step=32), input_shape=(X.shape[1],),
                    activation='relu', kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout1', 0.1, 0.5, step=0.1)))

    model.add(Dense(hp.Int('units2', min_value=64, max_value=512, step=32), activation='relu',
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout2', 0.1, 0.5, step=0.1)))

    model.add(Dense(hp.Int('units3', min_value=64, max_value=512, step=32), activation='relu',
                    kernel_regularizer=l2(0.001)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout3', 0.1, 0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(hp.Float('learning_rate', 1e-5, 1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [5]:
# Initialize Keras Tuner
tuner = RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=30,
                     executions_per_trial=3,
                     directory='my_dir',
                     project_name='pima_diabetes')

Reloading Tuner from my_dir\pima_diabetes\tuner0.json


In [6]:
# Prepare K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Lists to store results from each fold
fold_accuracies = []


In [7]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Search for the best hyperparameters
    tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test),
                 callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)])

    # Get the optimal hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    # Build the model with the optimal hyperparameters
    model = tuner.hypermodel.build(best_hps)

    # Define callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

    # Train the model with the optimal hyperparameters
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test),
              callbacks=[early_stopping, reduce_lr, checkpoint])

    # Load the best model
    model.load_weights('best_model.h5')

    # Evaluate the model
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    fold_accuracies.append(accuracy)

Epoch 1/100
20/20 [==============================] - 2s 19ms/step - loss: 1.2694 - accuracy: 0.6792 - val_loss: 1.0675 - val_accuracy: 0.6753 - lr: 0.0042
Epoch 2/100
20/20 [==============================] - 0s 6ms/step - loss: 1.1132 - accuracy: 0.7248 - val_loss: 1.0606 - val_accuracy: 0.6818 - lr: 0.0042


c:\BANGKIT\capstone\diabetters_capstone_project\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 3/100
20/20 [==============================] - 0s 6ms/step - loss: 1.0744 - accuracy: 0.7443 - val_loss: 1.0463 - val_accuracy: 0.6948 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9905 - accuracy: 0.7590 - val_loss: 1.0271 - val_accuracy: 0.7013 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9664 - accuracy: 0.7590 - val_loss: 1.0261 - val_accuracy: 0.6883 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 6ms/step - loss: 0.9314 - accuracy: 0.7638 - val_loss: 0.9906 - val_accuracy: 0.6883 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9013 - accuracy: 0.7769 - val_loss: 0.9704 - val_accuracy: 0.6883 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.8734 - accuracy: 0.7736 - val_loss: 0.9236 - val_accuracy: 0.7532 - lr: 0.0042
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8

c:\BANGKIT\capstone\diabetters_capstone_project\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 0s 7ms/step - loss: 1.0078 - accuracy: 0.7557 - val_loss: 1.0131 - val_accuracy: 0.7143 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9737 - accuracy: 0.7638 - val_loss: 0.9896 - val_accuracy: 0.7078 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9563 - accuracy: 0.7720 - val_loss: 0.9735 - val_accuracy: 0.7143 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9104 - accuracy: 0.7769 - val_loss: 0.9331 - val_accuracy: 0.7208 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 6ms/step - loss: 0.8707 - accuracy: 0.7866 - val_loss: 0.9163 - val_accuracy: 0.7403 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8489 - accuracy: 0.7769 - val_loss: 0.9055 - val_accuracy: 0.7662 - lr: 0.0042
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.8242 - accura

...

20/20 [==============================] - 0s 5ms/step - loss: 1.0119 - accuracy: 0.7606 - val_loss: 1.1181 - val_accuracy: 0.6039 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.9934 - accuracy: 0.7524 - val_loss: 1.0918 - val_accuracy: 0.6169 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9351 - accuracy: 0.7785 - val_loss: 1.0560 - val_accuracy: 0.6169 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9081 - accuracy: 0.7752 - val_loss: 1.0301 - val_accuracy: 0.6169 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8871 - accuracy: 0.7704 - val_loss: 1.0077 - val_accuracy: 0.6234 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8742 - accuracy: 0.7769 - val_loss: 0.9593 - val_accuracy: 0.6883 - lr: 0.0042
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8454 - accura

c:\BANGKIT\capstone\diabetters_capstone_project\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 0s 7ms/step - loss: 1.0213 - accuracy: 0.7626 - val_loss: 0.9950 - val_accuracy: 0.7451 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9917 - accuracy: 0.7642 - val_loss: 0.9783 - val_accuracy: 0.7516 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9487 - accuracy: 0.7463 - val_loss: 0.9403 - val_accuracy: 0.7712 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.9118 - accuracy: 0.7756 - val_loss: 0.9555 - val_accuracy: 0.7451 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8693 - accuracy: 0.7951 - val_loss: 0.9083 - val_accuracy: 0.7582 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.8749 - accuracy: 0.7821 - val_loss: 0.8880 - val_accuracy: 0.7516 - lr: 0.0042
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8150 - accura

c:\BANGKIT\capstone\diabetters_capstone_project\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 0s 5ms/step - loss: 1.0941 - accuracy: 0.7252 - val_loss: 1.0438 - val_accuracy: 0.6863 - lr: 0.0042
Epoch 4/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9780 - accuracy: 0.7610 - val_loss: 1.0230 - val_accuracy: 0.6993 - lr: 0.0042
Epoch 5/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9484 - accuracy: 0.7675 - val_loss: 1.0066 - val_accuracy: 0.6928 - lr: 0.0042
Epoch 6/100
20/20 [==============================] - 0s 7ms/step - loss: 0.9057 - accuracy: 0.7675 - val_loss: 0.9725 - val_accuracy: 0.7386 - lr: 0.0042
Epoch 7/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8996 - accuracy: 0.7756 - val_loss: 0.9412 - val_accuracy: 0.7451 - lr: 0.0042
Epoch 8/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8688 - accuracy: 0.7724 - val_loss: 0.9236 - val_accuracy: 0.7451 - lr: 0.0042
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.8035 - accura

In [8]:
# Calculate the mean accuracy across all folds
mean_accuracy = np.mean(fold_accuracies)
print(f'Mean Accuracy across all folds: {mean_accuracy}')

# Save the final model
model.save('diabetters.h5')
print("Final optimized model saved as 'diabetters.h5'")


Mean Accuracy across all folds: 0.7707919531448943
Final optimized model saved as 'diabetters.h5'


c:\BANGKIT\capstone\diabetters_capstone_project\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
# Load the saved model
from keras.models import load_model

# Load the final optimized model
final_model = load_model('diabetters.h5')

# Example input data for prediction (same preprocessing as training data)
example_input = np.array([[11,138,76,0,0,33.2,0.42,35]])
example_input = scaler.transform(example_input)

# Predict the outcome
example_prediction = final_model.predict(example_input)
predicted_class = (example_prediction > 0.5).astype("int32")

print(f'Predicted class for the example input: {predicted_class[0][0]}')

1/1 [==============================] - 0s 115ms/step
Predicted class for the example input: 1
